In [11]:
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import emoji
import plotly.express as px
from collections import defaultdict
import nltk

In [16]:
myfile = open('chat.txt')
content = myfile.read()
chat = defaultdict(list)
for line in content:
  try:
    person = line.split(':')[2][7:]
    text = nltk.sent_tokenize(':'.join(line.split(':')[3:]))
    chat[person].extend(text)
  except:
    pass

In [21]:
posts = nltk.corpus.nps_chat.xml_posts()

def extract_features(post):
  features={}
  for word in nltk.word_tokenize(post):
    features['contains({})'.format(word.lower())]=True
  return features

fposts = [(extract_features(p.text),p.get('class')) for p in posts]
test_size = int(len(fposts)*0.1)
train_set,test_set = fposts[test_size:],fposts[:test_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [41]:
df= pd.read_csv('chat.txt', names=['Date','Message','Col1','Col2'])
#map = df.T.plot(kind = 'bar', figsize=(7,5), legend = True, fontsize = 12, color=['b','g'])
df.head()

,Date,Message,Col1,Col2
0,[2021-02-21,10:59:44 PM] Harsh Patel Computer Science: ‎M...,not even WhatsApp,can read or listen to them.
1,[2021-02-21,10:59:44 PM] Harsh Patel Computer Science: Hi...,NaN,NaN
2,[2021-02-22,1:50:19 AM] Krishna: Hi! Nice!,NaN,NaN
3,[2021-02-22,2:09:09 AM] Harsh Patel Computer Science: Do ...,NaN,NaN
4,[2021-02-22,2:10:08 AM] Harsh Patel Computer Science: I m...,NaN,NaN
